# Financial Instruments - Homework 1

## Task 1: Arbitrage and Forward Rates

Consider a one-year forward contract for converting between dollars and Euros. The current exchange rate is $S_0 = 1.20$ for each Euro. The one-year risk-free rate in dollars is $r_{USD} = 5\%$ in continuously compounded units. The one-year risk-free rate in Euros is $r_{EUR} = 4.5\%$ in continuously compounded units.

### (1) According to the principle of no-arbitrage, what should be the one-year forward rate?

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Parameters
S0 = 1.20
r_usd_cc = 0.05
r_eur_cc = 0.045
T = 1.0

# Formula: F = S * exp((r_usd - r_eur) * T)
F_theoretical = S0 * np.exp((r_usd_cc - r_eur_cc) * T)
print(f"Theoretical Forward Rate: {F_theoretical:.6f}")

The theoretical forward rate is calculated as:
$$
F = S_0 e^{(r_{USD} - r_{EUR})T} = 1.20 e^{(0.05 - 0.045) \times 1} = 1.20 e^{0.005} \approx 1.206015
$$

### (2) Suppose that the one-year forward contract is currently trading at $1.15 per Euro. Is there an arbitrage opportunity? If so, explain in detail the trading you would like to do to exploit this arbitrage opportunity.

In [ ]:
F_market = 1.15
if F_market != F_theoretical:
    print(f"Arbitrage Opportunity exists. Market: {F_market}, Theoretical: {F_theoretical:.6f}")
    if F_market < F_theoretical:
        print("Strategy: Buy Forward (Long EUR), Sell Spot (Short EUR)")
    else:
        print("Strategy: Sell Forward (Short EUR), Buy Spot (Long EUR)")

Yes, there is an arbitrage opportunity because $F_{market} = 1.15 \neq F_{theoretical} \approx 1.206$.

Since $F_{market} < F_{theoretical}$, the forward is "too cheap". We should Buy the forward.

**Arbitrage Strategy:**
1. **Borrow EUR** at $r_{EUR} = 4.5\%$. Borrow $e^{-0.045}$ EUR today.
2. **Convert to USD** at Spot Rate $S_0 = 1.20$.
3. **Invest USD** at $r_{USD} = 5\%$.
4. **Enter Long Forward Contract** to buy EUR at $F_{market} = 1.15$ in 1 year.

At Maturity ($T=1$):
- USD investment grows to: $1.20 \times e^{0.05} \approx 1.2615$ USD.
- Use Forward to buy 1 EUR (repaying the loan which grew to 1 EUR if we borrowed $e^{-0.045}$) for $1.15$ USD.
- Profit: $1.2615 - 1.2029 = 0.0586$ USD.

## Task 2: Forward Rates and Covered Interest Rate Parity

### (1) For each date, use the Forward Rate formula... to compute the Forward Exchange Rate...

We load the data and perform the calculations.

In [ ]:
# Load Data
df_raw = pd.read_excel("Assignments/Assignment 1/DataHW1.xls", header=None)

# Extract Data
data = df_raw.iloc[2:].copy()
data.columns = [
    'Date', 'Spot',
    'Fwd_1M', 'Fwd_3M', 'Fwd_6M', 'Fwd_1Y',
    'USL_1M', 'USL_3M', 'USL_6M', 'USL_1Y',
    'EUL_1M', 'EUL_3M', 'EUL_6M', 'EUL_1Y'
]

data = data.dropna(subset=['Date'])
data = data[data['Date'] != 'Data Source: Bloomberg']
data.reset_index(drop=True, inplace=True)

# Convert types
cols_to_numeric = data.columns[1:]
for col in cols_to_numeric:
    data[col] = pd.to_numeric(data[col])

maturities = {
    '1M': 1/12,
    '3M': 3/12,
    '6M': 6/12,
    '1Y': 1.0
}

results = []

for idx, row in data.iterrows():
    date_val = row['Date']
    spot = row['Spot']
    
    row_res = {'Date': date_val, 'Spot': spot}
    
    for m in ['1M', '3M', '6M', '1Y']:
        t = maturities[m]
        
        r_us_L = row[f'USL_{m}'] / 100.0
        r_eu_L = row[f'EUL_{m}'] / 100.0
        
        # Convert to Continuously Compounded
        r_us_cc = np.log(1 + r_us_L * t) / t
        r_eu_cc = np.log(1 + r_eu_L * t) / t
        
        # Calculate Forward Rate
        fwd_calc = spot * np.exp((r_us_cc - r_eu_cc) * t)
        
        fwd_mkt = row[f'Fwd_{m}']
        
        row_res[f'Fwd_{m}_Calc'] = fwd_calc
        row_res[f'Fwd_{m}_Mkt'] = fwd_mkt
        row_res[f'Diff_{m}'] = fwd_calc - fwd_mkt
        
    results.append(row_res)

df_results = pd.DataFrame(results)
df_results.head()

### (2) Do your forward exchange rates match (approximately) the quoted ones?

In [ ]:
cols_diff = ['Date', 'Diff_1M', 'Diff_3M', 'Diff_6M', 'Diff_1Y']
print(df_results[cols_diff])

fig = go.Figure()
fig.add_trace(go.Bar(
    x=df_results['Date'],
    y=df_results['Diff_1Y'],
    name='Diff (Calc - Mkt) 1Y'
))
fig.update_layout(title='Difference between Calculated and Market Forward Rates (1Y)',
                  yaxis_title='Difference')
fig.show()

### (3) If not, pick one particular date in which the parity is violated, and describe the arbitrage strategy you would undertake.

On 2008-10-01, there is a visible discrepancy.

In [ ]:
row_2008 = df_results[df_results['Date'] == '2008-10-01'].iloc[0]
print(row_2008[['Date', 'Fwd_1Y_Calc', 'Fwd_1Y_Mkt', 'Diff_1Y']])

On **2008-10-01**, the Market Forward ($1.396$) is significantly higher than the Calculated/Theoretical Forward ($1.383$).
The Market is overvaluing the Forward (too expensive).

**Strategy: Cash and Carry (Synthetically Create Forward Long, Sell Actual Forward Short)**
1. **Borrow USD** at US LIBOR.
2. **Convert to EUR** at Spot.
3. **Invest EUR** at EUR LIBOR.
4. **Sell EUR Forward** at Market Price.